In [29]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string
from string import digits
from nltk.corpus import stopwords
ps = PorterStemmer()
from nltk.stem import PorterStemmer

In [30]:
from datetime import datetime

def convert_dt(s):
    return datetime.strptime(s, '%m/%d/%Y')

def get_date(d):
    return int(d.days)

In [ ]:
ps.stem('opening')

In [31]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            p  = (lemmatizer.lemmatize(w))
            str_ += ' '  + ps.stem(p)
    
    return str_.strip()

In [ ]:
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

model = loadGloveModel('/home/prashant/Downloads/kagsa/data/glove.840B.300d.txt')
#vocab = model.keys()

In [ ]:
def get_vector(model,all_data):
    vector_sen = []
    for d in all_data:
        single_sen_vec = []
        words = d.split(' ')
        vec_created =  0.0
        for w in words:
            try:
                get_word_vec = model[w]
                vec_created += 1.0
            except:
                pass
        if vec_created / len(words) > .5: 
            single_sen_vec.append(get_word_vec)
            v = np.array(single_sen_vec).mean(axis=0)
            vector_sen.append(v)
            sent =1
        else:
            vector_sen.append(np.zeros(300))
            sent = 0 
    return (vector_sen)

In [35]:
df_less = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
df_less['tag'] = 'train'


In [36]:
df_less.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes,Consumer-complaint-summary,tag
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Closed with explanation,Yes,"Seterus, Inc a déposé un faux rapport auprès d...",train
1,Tr-2,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,Closed with non-monetary relief,No,XX / XX / XXXX La requête en faillite n ° XXXX...,train
2,Tr-3,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,Closed with explanation,No,"El XXXX / XXXX / 15, estaba preparando el vuel...",train
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,Closed with explanation,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea...",train
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,Closed with explanation,No,J'ai obtenu un compte de crédit de soins pour ...,train


In [33]:
df_more =  pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/more_data.csv')

In [34]:
df_more.head()

,product,company_response_to_consumer
0,Mortgage,Closed with explanation
1,Mortgage,Closed with explanation
2,Credit reporting,Closed with explanation
3,Student loan,Closed with explanation
4,Debt collection,Closed with explanation


In [38]:
df_less = df_less[['Transaction-Type','Complaint-Status']]

In [37]:
df_more.columns = ['Transaction-Type','Complaint-Status']

In [39]:
df_train = pd.concat([df_more,df_less],axis =0)

In [40]:
df_test = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/test.csv') 
df_test['tag'] = 'test'
print df_test.shape

(18543, 9)


In [ ]:
df_train.head()

In [41]:
df_train['Transaction-Type_pre'] = df_train['Transaction-Type'].apply(preprocess_data)
print 'train-- done'
df_test['Transaction-Type_pre'] = df_test['Transaction-Type'].apply(preprocess_data)
print 'test -- done'

train-- done
test -- done


In [42]:
df_train['Transaction-Type_pre'] = df_train['Transaction-Type_pre'].map(lambda x : x.replace(' ','_'))
df_test['Transaction-Type_pre'] = df_test['Transaction-Type_pre'].map(lambda x : x.replace(' ','_'))

In [65]:
df_train.tail(10)

,Transaction-Type,Complaint-Status,Transaction-Type_pre
43256,Mortgage,Closed with explanation,mortgag
43257,Credit card,Closed with explanation,credit_card
43258,Debt collection,Closed with explanation,debt_collect
43259,Mortgage,Closed with explanation,mortgag
43260,"Credit reporting, credit repair services, or o...",Closed with explanation,credit_report_credit_repair_servic_person_cons...
43261,"Credit reporting, credit repair services, or o...",Closed with explanation,credit_report_credit_repair_servic_person_cons...
43262,Debt collection,Closed with explanation,debt_collect
43263,Mortgage,Closed with explanation,mortgag
43264,Mortgage,Closed with explanation,mortgag
43265,Debt collection,Closed with explanation,debt_collect


In [11]:
# df_train['Complaint-reason'] = df_train['Complaint-reason'].apply(preprocess_data)
# print 'train-- done'
# df_test['Complaint-reason'] = df_test['Complaint-reason'].apply(preprocess_data)
# print 'test -- done'

train-- done
test -- done


In [ ]:
# df_train['Consumer-complaint-summary_pre'] = df_train['Consumer-complaint-summary'].apply(preprocess_data)
# print 'train-- done'
# df_test['Consumer-complaint-summary_pre'] = df_test['Consumer-complaint-summary'].apply(preprocess_data)
# print 'test -- done'

In [ ]:
# df_train['Complaint-Status'].value_counts()

In [ ]:
# df_train['Consumer-complaint-summary_pre'] = df_train['Consumer-complaint-summary_pre'].map(lambda x : re.sub("x+","",x))
# df_test['Consumer-complaint-summary_pre'] = df_test['Consumer-complaint-summary_pre'].map(lambda x : re.sub("x+","",x))

In [ ]:
# df_train['Date-received'] = df_train['Date-received'].apply(convert_dt)
# df_test['Date-sent-to-company'] = df_test['Date-sent-to-company'].apply(convert_dt)

In [ ]:
# df_train['Date-sent-to-company'] = df_train['Date-sent-to-company'].apply(convert_dt)
# df_test['Date-received'] = df_test['Date-received'].apply(convert_dt)

In [ ]:
# df_train['date_sub'] = (df_train['Date-sent-to-company'] -  df_train['Date-received'])
# df_test['date_sub'] = (df_test['Date-sent-to-company'] -  df_test['Date-received'])

# df_train['date_sub'] = df_train['date_sub'].apply(get_date) 
# df_test['date_sub'] = df_test['date_sub'].apply(get_date) 

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

text = list(df_train['Transaction-Type_pre'])
text.extend(list(df_test['Transaction-Type_pre']))


vecto = TfidfVectorizer(min_df=1)

# tokenize and build vocab
vecto.fit(text)

fet = df_train['Transaction-Type_pre']
# encode document
vectors = vecto.transform(fet)
# summarize encodehd vector
print(vectors.shape)
#print(vector.toarray())


(590155, 18)


In [44]:
# vectors = get_vector(model,list(df_train['Consumer-complaint-summary_pre']))
# vectors_test =  get_vector(model,list(df_test['Consumer-complaint-summary_pre']))
from sklearn.feature_extraction.text import TfidfVectorizer

text = (df_test['Transaction-Type_pre'])

# encode document
vectors_test = vecto.transform(text)
# summarize encoded vector
print(vectors_test.shape)
#print(vector.toarray())


(18543, 18)


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

text = list(df_train['Complaint-reason'])
text.extend(list(df_test['Complaint-reason']))


vectorizer = TfidfVectorizer(min_df=5)

# tokenize and build vocab
vectorizer.fit(text)

fet = df_train['Complaint-reason']
# encode document
vector_cr = vectorizer.transform(fet)
# summarize encoded vector
print(vector_cr.shape)
#print(vector.toarray())


In [ ]:
df_

In [46]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# text = (df_test['Complaint-reason'])

# # encode document
# vector_cr_test = vectorizer.transform(text)
# # summarize encoded vector
# print(vector_cr_test.shape)
# #print(vector.toarray())


In [48]:
# vec_cr_df = pd.DataFrame(vector_cr.toarray())
# vec_cr_df_test = pd.DataFrame(vector_cr_test.toarray())

# print vec_cr_df.shape
# print vec_cr_df_test.shape

In [50]:
vec_cs_df = pd.DataFrame(vectors.toarray())
vec_cs_df_test =  pd.DataFrame(vectors_test.toarray())
print vec_cs_df.shape
print vec_cs_df_test.shape

(590155, 18)
(18543, 18)


In [51]:
ndf = pd.concat([vec_cs_df],axis =1)
ndf.shape

(590155, 18)

In [ ]:
# import time
# from sklearn.manifold import TSNE
# n_sne = 7000
# time_start = time.time()
# tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
# ndf = tsne.fit_transform(ndf)
# print 'train done'

# import time
# from sklearn.manifold import TSNE
# n_sne = 7000 
# time_start = time.time()
# tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
# ndf_test = tsne.fit_transform(ndf_test)

In [52]:
ndf_test = pd.concat([vec_cs_df_test],axis =1)
ndf_test.shape

(18543, 18)

In [ ]:
# ndf = pd.DataFrame(ndf)
# ndf_test = pd.DataFrame(ndf_test)


In [ ]:
# #Finding number of parameters
# from sklearn.decomposition import PCA
# #Fitting the PCA algorithm with our Data
# pca = PCA().fit(ndf)

In [ ]:
# #Plotting the Cumulative Summation of the Explained Variance
# import matplotlib.pyplot as plt 
# %matplotlib inline
# plt.figure()
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.xlabel('Number of Components')
# plt.ylabel('Variance (%)') #for each component
# plt.title('Pulsar Dataset Explained Variance')
# plt.show()

In [ ]:
# # ndf['Transaction-Type'] = df_train['Transaction-Type_pre']
# # ndf_test['Transaction-Type'] = df_test['Transaction-Type_pre']
# pca = PCA(n_components=75)
# ndf = pca.fit_transform(ndf)

In [ ]:
# ndf_test = pca.fit_transform(ndf_test)

In [53]:
# # ndf['date_sub'] = df_train['date_sub']
# # ndf_test['date_sub'] = df_test['date_sub']
ndf = pd.DataFrame(ndf)
ndf_test =  pd.DataFrame(ndf_test)

In [54]:
ndf['Complaint-Status'] = list(df_train['Complaint-Status'])
#ndf_test['Complaint-Status'] = df_test['Complaint-Status']

In [ ]:
#ndf['Consumer-disputes'] = df_train['Consumer-disputes']
#ndf_test['Consumer-disputes'] = df_test['Consumer-disputes']

In [ ]:
# ndf = pd.get_dummies(ndf , columns=['Transaction-Type'])

In [ ]:
# ndf_test = pd.get_dummies(ndf_test, columns=['Transaction-Type'])

In [55]:
y_train = ndf['Complaint-Status']
x_train = ndf.drop(['Complaint-Status'],axis =1)

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train , y_train)

In [66]:
from sklearn.multiclass import OneVsRestClassifier
# # Import the model we are using
from sklearn.ensemble import RandomForestClassifier

# # Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 200, random_state = 42)
#rf = MultinomialNB()
# # Train the model on training data


gnb = OneVsRestClassifier(rf).fit(X_train, y_train_internal)
#pred_internal = gnb.predict(X_test)

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# neigh = KNeighborsClassifier(n_neighbors=3)
# gnb = OneVsRestClassifier(neigh).fit(x_train, y_train)

In [ ]:
# from sklearn.linear_model import SGDClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
# text_clf_svm = Pipeline([('clf-svm', SGDClassifier(loss='hinge', penalty='l2',\
# alpha=1e-3, n_iter=50, random_state=42))])
# _ = OneVsRestClassifier(text_clf_svm.fit(x_train, y_train))
# #predicted_svm = text_clf_svm.predict(twenty_test.data)

In [58]:
pred_internal = gnb.predict(X_test)

In [ ]:
# # Import the model we are using
# from sklearn.ensemble import RandomForestClassifier

# # Instantiate model with 1000 decision trees
# rf = RandomForestClassifier(n_estimators = 200, random_state = 42)

# # Train the model on training data
# rm = rf.fit(X_train, y_train_internal)
# pred_internal = rm.predict(X_test)

In [59]:
import sklearn
sklearn.metrics.f1_score(pred_internal,y_test_internal,average='weighted')

0.8524840070777188

In [60]:
id_ = list(df_test['Complaint-ID'])

In [25]:
# y_pred_prob = rm.predict_proba(ndf_test)
df_test.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary,tag
0,Te-1,8/18/2016,Bank account or service,account open close manag,Company has responded to the consumer and the ...,8/18/2016,No,XXXX / XXXX / 16 I called Citibank to open a c...,test
1,Te-2,4/18/2016,Debt collection,commun tactic,Company believes it acted appropriately as aut...,4/20/2016,No,I'm struggling financially. I called and I off...,test
2,Te-3,3/23/2016,Credit reporting,incorrect inform credit report,NaN,3/23/2016,No,"In XXXX of 2015, an automatic payment was conf...",test
3,Te-4,6/26/2017,Student loan,deal lender servic,NaN,6/26/2017,NaN,"I submitted a request to XXXX, which is my cur...",test
4,Te-5,5/13/2016,Credit reporting,incorrect inform credit report,Company has responded to the consumer and the ...,5/13/2016,No,A state tax lien was filed against me XXXX / X...,test


In [61]:
y_pred = gnb.predict(ndf_test)

In [ ]:
# tune_pred = []

# for cnt,p in enumerate(y_pred_prob): 
#     if p[4] > .1:
#         tune_pred.append('Untimely response')
#         continue
#     if p[0] > .1:
#         tune_pred.append('Closed')
#         continue

#     tune_pred.append(y_pred[cnt])

In [ ]:
# rm.classes_
# print ''

In [62]:
result = pd.DataFrame(id_,columns=['Complaint-ID'])
result['Complaint-Status'] = y_pred

In [63]:
result.to_csv('/home/prashant/Downloads/c3cc8568-0-dataset/jugsa2.csv',index=None)

In [ ]:
(result.loc[result['Complaint-Status'] == 'Closed with explanation' ])

In [ ]:
result['cr'] = df_test['Complaint-reason']

In [ ]:
r

In [ ]:
df_test['Consumer-disputes'].value_counts()

In [ ]:
df_train['Consumer-disputes'].value_counts()